# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [371]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2 and all(isinstance(arg, int) for arg in args):
            self.n = args[0]  # number of rows
            self.m = args[1]  # number of columns
            self.data = [[0 for _ in range(self.m)] for _ in range(self.n)]
        elif len(args) == 1 and isinstance(args[0], list):
            if all(isinstance(row, list) for row in args[0]):
                row_lengths = [len(row) for row in args[0]]
                if len(set(row_lengths)) != 1:
                    raise ValueError("All rows must have the same number of columns")
                self.n = len(args[0])  # number of rows
                self.m = len(args[0][0])  # number of columns
                self.data = args[0]
            else:
                raise ValueError("Matrix should be a list of lists")
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, pos):
        if isinstance(pos, tuple) and len(pos) == 2:
            i, j = pos
            return self.data[i][j]
        elif isinstance(pos, int):
            return self.data[pos]
        else:
            raise IndexError("Invalid index")

    def __setitem__(self, pos, value):
        if isinstance(pos, tuple) and len(pos) == 2:
            i, j = pos
            self.data[i][j] = value
        elif isinstance(pos, int):
            self.data[pos] = value
        else:
            raise IndexError("Invalid index")

    def __repr__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.data])

    def __eq__(self, other):
        return self.n == other.n and self.m == other.m and self.data == other.data

    def assign(self, other):
        if isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                self.data = [row[:] for row in other.data]
            else:
                raise ValueError("Matrices must be of the same size")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.data = [row[:] for row in other]
            else:
                raise ValueError("Matrix must match dimensions")

    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        transposed_matrix = list(map(list, zip(*self.data)))
        return Matrix(transposed_matrix)

    def row(self, n):
        return Matrix([self.data[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.data])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = [row[m_0:m_1] for row in self.data[n_0:n_1]]
        return Matrix(block_matrix)

    def __add__(self, other):
        result = [[self.data[i][j] + other.data[i][j] for j in range(self.m)] for i in range(self.n)]
        return Matrix(result)

    def __sub__(self, other):
        result = [[self.data[i][j] - other.data[i][j] for j in range(self.m)] for i in range(self.n)]
        return Matrix(result)

    def scalar_mul(self, c):
        result = [[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)]
        return Matrix(result)

    def __rmul__(self, other):
        return self.scalar_mul(other)

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.__matmul__(other)
        else:
            return self.scalar_mul(other)

    def __matmul__(self, other):
        result = [[sum(self.data[i][k] * other.data[k][j] for k in range(self.m)) for j in range(other.m)] for i in range(self.n)]
        return Matrix(result)

    def __element_mult__(self,other):
        result = [[self.data[i][j] * other.data[i][j] for j in range(self.m)] for i in range(self.n)]
        return Matrix(result)

def constant(n, m, c):
    return [[float(c) for _ in range(m)] for _ in range(n)]

def zeros(n, m):
    return [[0 for _ in range(m)] for _ in range(n)]

def ones(n,m):
    return [[1 for _ in range(m)] for _ in range(n)]
    

In [348]:
m1=Matrix(2,3)
m2= Matrix([[1, 2, 3], [4, 5, 6]])
print(m1)

0 0 0
0 0 0


In [349]:
m1.assign(m2)
print(m1)

1 2 3
4 5 6


In [350]:
m1.shape()

(2, 3)

In [351]:
m1.transpose()

1 4
2 5
3 6

In [352]:
m1.to_list()

[[1, 2, 3], [4, 5, 6]]

In [353]:
m1.block(1,2,1,2)

5

In [354]:
constant(3,5,2)

[[2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0]]

In [355]:
zeros(3,4)

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [356]:
ones(3,4)

[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]

In [193]:
m1.scalarmul(2)

2
8
4
10
6
12

In [194]:
print(m1)

1 2 3
4 5 6


In [195]:
print(m2)

1 2 3
4 5 6


In [196]:
m1.add(m2)

2 4 6
8 10 12

In [197]:
m1.sub(m2)

0 0 0
0 0 0

In [281]:
m1.mat_mul(m3)

AttributeError: 'Matrix' object has no attribute 'mat_mul'

In [199]:
m1.element_mult(m2)

1 4 9
16 25 36

In [200]:
m1.equals(m2)

True

In [357]:
m1+m2

2 4 6
8 10 12

In [358]:
m1-m2

0 0 0
0 0 0

In [359]:
2*m1

2 4 6
8 10 12

In [360]:
m1 == m2

True

In [361]:
m3 = Matrix([[1,2],[3,4],[5,6]])

In [364]:
A = Matrix([[1,2], [3,4]])
B = Matrix([[5,6],[7,8]])
C = Matrix([[3,4],[5,6]])

In [376]:
product_1 = A*B
result_1= product_1*C
product_2= B*C
result_2 = product_2*A

result_1==result_2

TypeError: 'Matrix' object is not callable

In [372]:
sum_1= B+C
result_A = A*sum_1
product_a = A*B
product_b= A*C
result_B=product_a + product_b
result_A==result_B

TypeError: 'Matrix' object is not callable

In [373]:
AB=A*B
BA=B*A
AB==BA

False

In [374]:
I = Matrix([[1, 0], [0, 1]])

In [375]:
AI= A*I
AI==A

TypeError: 'Matrix' object is not callable